# NLE Assessed Coursework 3: Question 1

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 1 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=184521 #this MUST be updated to your candidate number so that you get a unique data sample


In [2]:
#preliminary imports
import sys
sys.path.append(r'resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools impor

t zip_longest
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
import random
from nltk.corpus import stopwords
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import gutenberg

Sussex NLTK root directory is resources


## Question 1: Part-of-Speech Tagging (25 marks)


In [3]:
#Do NOT change the code in this cell.

#preparing corpus
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def tag_sent(asent):
    tagged=[]
    for t in asent:
        tagged.append((t.lower_,t.pos_))
    return tagged

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()

def display_tags(sentslist):
    taglist={}
    for s in sentslist:
        for t in s:
            form=t.lower_
            pos=t.pos_
            taglist[pos]=taglist.get(pos,0)+1
    print(len(taglist.keys()))
    print(taglist)
        
def get_train_test(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=10
    train=[tag_sent(s) for s in sentslist[testsize:]]
    test=[tag_sent(s) for s in sentslist[:testsize]]
    return train,test
    
alice=clean_text(gutenberg.raw('carroll-alice.txt'))
nlp_alice=list(nlp(alice).sents)

The code below will generate (unique to you) training and test sets of pos-tagged sentences from the novel "Alice in Wonderland"

In [7]:
#do not change the code in this cell
allsents=list(nlp_alice)
train,test=get_train_test(allsents)

In the cell below, a class `unigram_tagger` is defined, which will be used in parts a and b of this question

In [8]:
#do not change the code in this cell
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                

a) Create an instance of the `unigram_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).
Explain what is stored, after training, in the three instance variables:
* `self.tagcounts`
* `self.wordcounts`
* `self.tagperwordcounts`

You should refer to the code and specific examples from the training data. \[4 marks \]

In [99]:
Tagger= unigram_tagger(train)
trained=unigram_tagger.train(Tagger,train)

The .tagcounts variable counts the total number of occurences that a certain tag that has been applied to words. For example with my training data there are 9222 words tagged as "VERB" as well as 2782 tagged as "ADJ" aka adjectives. This variable stores the number of tags for many different tags such as pronouns, numbers and punctuation, in total there are 16 different tags that are counted in my data. 

The .wordcounts variable is very similar to .tagcounts but instead of counting the tags of each word it counts the occurence of each word in the data set, for example "i" appears 1066 times but "footman" only appears 28 times. This variable does however have some fatal flawws as it counts punctuation such as "'" and suffix's such as "'ll" which are not words and therefore shouldn't necessarily be counted as such.

The .tagperwordcounts variable takes the previous two variables and combines them into one by taking each word and counting how many times each tag is applied to it. For example "i" is only tagged with "PRON" aka pronoun and this occurs 1066 times but "no" is tagged as "INTJ", "DET" and "ADV" with their totals being 40, 126 and 14 respectively.

b) In the cells below, **extend** the code for the `unigram_tagger` class so that it also has a `tag()` method.  This method should assign the tag, $t$, which maximises the unigram tag probability, $P(t|w)$, for the observed word, $w$.  **Evaluate** the performance of the `unigram_tagger` on your test sentences.  **Discuss** your results. \[8 marks\] 



In [100]:
import operator
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
    
    def tag(self,data):
        words_tagged = []
        for sentence in data:
            for word in sentence:
                if (word[0] in self.tagperwordcounts):
                    dist = self.tagperwordcounts[word[0]]
                    ordered = sorted(list(dist.items()), key = operator.itemgetter(1), reverse = True)
                    words_tagged.append((word[0],ordered[0][0]))
                else:
                    words_tagged.append((word,'NONE'))
        return words_tagged
            
        

In [101]:
tagger= unigram_tagger(train)
tagged=tagger.tag(test)
print(tagged)

[('said', 'VERB'), ('alice', 'PROPN'), ('indignantly', 'ADV'), ('.', 'PUNCT'), ("'", 'PUNCT'), ('please', 'INTJ'), (',', 'PUNCT'), (("ma'am", 'PROPN'), 'NONE'), (',', 'PUNCT'), ('is', 'AUX'), ('this', 'DET'), ('new', 'ADJ'), (('zealand', 'PROPN'), 'NONE'), ('or', 'CCONJ'), (('australia', 'PROPN'), 'NONE'), ('?', 'PUNCT'), ("'", 'PUNCT'), ('*', 'PUNCT'), ('here', 'ADV'), ('the', 'DET'), ('dormouse', 'PROPN'), ('shook', 'VERB'), ('itself', 'PRON'), (',', 'PUNCT'), ('and', 'CCONJ'), ('began', 'VERB'), ('singing', 'VERB'), ('in', 'ADP'), ('its', 'DET'), ('sleep', 'NOUN'), ("'", 'PUNCT'), ('twinkle', 'PROPN'), (',', 'PUNCT'), ('twinkle', 'PROPN'), (',', 'PUNCT'), ('twinkle', 'PROPN'), (',', 'PUNCT'), ('twinkle--', 'PROPN'), ("'", 'PUNCT'), ('and', 'CCONJ'), ('went', 'VERB'), ('on', 'ADP'), ('so', 'ADV'), ('long', 'ADJ'), ('that', 'DET'), ('they', 'PRON'), ('had', 'AUX'), ('to', 'PART'), ('pinch', 'VERB'), ('it', 'PRON'), ('to', 'PART'), ('make', 'VERB'), ('it', 'PRON'), ('stop', 'VERB'), ('

My trained tagger is very good at tagging the test data correctly however there are one words which could not be tagged as they were not in the training data. Out of the 60 entries there are only 8 which could not be tagged by the tagger and were given the tag of 'NONE', these being blame, shrimp, eels, soles, evdently, Australia, Zealand and ma'am. The majority of these are nouns with only one being an adverb, this makes sense as nouns are context specific and as Alice in Wonderland has nothing to do with Australia or New Zealand there will be no way for the tagger to know what to tag them.

Another error that occured was with 'twinkle' which is classified as a Proper Noun although it is more commonly associated with being a verb than a noun this error also also occurs with the verb sleep which can be seen as a "NOUN" but is more commonly seen as a "VERB". One of these cases of 'twinkle' also has the punctuation of '--' attached to the end which should've been split and tagged appropriately as a "PUNCT", punctuation.  

Other than these listed errors and non-tagables the tagger performed as expected giving it a 82% success rate and therefore making it very successful. The main way to improve this would be to widen the training data to be less focused on one thing aka Alice in Wonderland and allow it to be able to process more nouns with more accuracy.

In [102]:
#do not change the code in this cell
class hmm_tagger():
    
    def __init__(self,trainingdata=[]):
        
        self.emissions={}
        self.transitions={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            previous="START"
            for token,tag in sentence:
                
                current=self.emissions.get(tag,{})
                current[token]=current.get(token,0)+1
                self.emissions[tag]=current
                bigram=self.transitions.get(previous,{})
                bigram[tag]=bigram.get(tag,0)+1
                self.transitions[previous]=bigram
                previous=tag
                
        self.emissions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.emissions.items()}
        self.transitions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.transitions.items()}

c) **Create** an instance of the `hmm_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).  With reference to your testing sentences (stored in `test`), **explain**
* how to calculate the probability of an observed sequence of words for a given sequence of tags: $$P(w_1^n|t_1^n)$$
* how to calculate the probablity of a possible sequence of tags for a given sequence of words: $$P(t_1^n|w_1^n)$$

\[6 marks\]

In [103]:
hmm=hmm_tagger(train)
trained_hmm=hmm.train(train)

To calculate the probabilities of an observed sequence of words the emission of each word must be calculated per tag, this is done by iterating through each word and its tag in the given list and counting them as such, in this case the list would be 'test'. The number of times a word is with a particular tag is then divided by the total number of times a word appears overall giving the probability that a word will be of a particular tag. This is then returned as a dictionary of a dictionary with the key of the first dictionary being the given tag, the key of the second being the word and its value being the probability that the word is of that particular tag. 
In the example of test, with the word 'said' there will be a probability of 1 of it being tagged with 'verb' as said appears twice and both times it is a 'verb'. Using these probabilities it is then possible to predict the probability of a sequence of words such as 'Alice said that'. To do so we look at what each word would need to be tagged to get a the sequence shown, being Proper Noun, Verb and Determiner, and then find the probability that each word would be tagged with the needed tag and multiply them together. This will give the final probability of the sequence occuring being 0.2656776803776148 * 0.09867707655606085 * 0.04755205402363533 giving a 0.0012466340703557 probability of this sequence occuring.

To calculate the probability of a possible sequence of tags you must use both emission and transition probabilities by applying Bayes Law. To do this you must get the given sequence of words's tags, so using our previous example of 'alice said that' would give us Proper Noun, Verb and Determiner. Using these values we can then get the transition probabilities for the tags and apply them to Bayes Law to get the final probability of the sequence.
To get the transition probabilities we must get each tag and count the occurences of each of its previous tags, once we have these values we take them and apply them to Bayes Rule e.g. 𝑡 ̂_1^𝑛=argmax┬(𝑡_1^𝑛 )⁡(𝑃(𝑤_1^𝑛│𝑡_1^𝑛 )𝑃(𝑡_1^𝑛) for each word and tag combination. This gives us the equation (0.0615229*0.265677) * (0.03979*0.098677) * (0.0351914*0.047552) giving us the final probability of 1.073954707625416e-7.

d) Using one of your `test` sentences as an example, **explain** how the Viterbi algorithm can be used to find the most likely tag sequence.  You do not need to write code for this question but should explain the calculations that need to be made at each step.  **Comment** on whether the sequence found by the Viterbi algorithm is correct for your chosen test sentence.  **Discuss** why using the Viterbi algorithm is better than the brute-force approach of enumerating and testing all tag sequence possibilities.\[7 marks\]


The Viterbi algorithm is a simple algorithm that is utilised to find the best possible path/solution for a problem by starting with the second item of a sequence and working out the optimal value before it, before moving onto the following items in the sequence. 

For example, we may can take a sentence from 'test' such as 'said alice indignantly', using this we can look at what would be the most likely tag to proceed each of these and therefore construct the most likely tag sequence to go from one to the each other. For this example the three tags that'll be considered will be Verb, ProperNoun and Adjective as they're the three that make up the sentence. Their probabilities for starting the sentence are V:0.114, PN:0.062 and A:0.007 and their probabilities for being assigned to 'Said' are V:0.0987, PN: 0.0007 and A:0. Multiplying these values to gether gives us the first values for this problem being 0.0113 for it being Verb and 0.0004 for it being ProperNoun and 0 for it being an Adjective. Taking this into account means that we will take verb as the starting point and move on to Alice's probabilities of being assigned Verb, ProperNoun and Adjective being V:0, PN:0.266 and A:0. The next stage is to go through all nine possibilities of which tag is most likely to proceed each tag and multiply this value by the probability of the tag being assigned to the word. Using this we can then determine the most likey tag to be assigned to 'Alice'. After these calculations the best method is a ProperNoun being assigned to 'Alice' following Verb that was assigned to 'said'. This process then continues for 'indignantly' and then for the end.

After all this the optimal solution is unsuprisingly Verb, ProperNoun, Adjective with a probability of 4.2824e-9 of this sequence occuring.

This method is better than brute forcing methods as you don't have to consider everysingle possibility as if you come across a tag that has 0 probability of being assigned to a word then that is three routes that don't have to be considered, in the case of my example. This therefore lowers the running the time of the algorithm and makes its execution much more efficent especially when you don't have to test each combination to check whether it works.